In [1]:
import SimpleITK as sitk
import numpy as np
from PIL import Image
import os

# Configuration
dir_location = 'refine'

# Set base paths
if dir_location.lower()=='internal':
    base_path = r'C:\Senior_Design\database\tablet_dataset'
elif dir_location.lower()=='external':
    base_path = r'D:\Senior_Design\database\tablet_dataset'
elif dir_location.lower()=='cloud':
    base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files\database\tablet_dataset'
elif dir_location.lower() == 'refine':
    base_path = r'D:\Darren\Files\database\tablet_dataset'
else:
    raise ValueError('Invalid directory location type')

# More paths
general_mhd_path = os.path.join(base_path, 'filler', 'mhd')
general_tiff_path = os.path.join(base_path, 'filler', 'tiff')

# Specify subfolder paths
tablets = [
    # "1_Microsphere",
    # "2_Tablet": "2_Tablet",
    # "3_SprayDriedDispersion",
    # '4_GenericD12',
    # '5_ClaritinD12',
    '6_30pWTMCC',
]

In [4]:
# Process each folder
for tablet in tablets:
    # Process "grayscale" and "segmented" subfolders
    for subfolder in ["grayscale", "segmented"]:
        subfolder_mhd_path = os.path.join(general_mhd_path.replace('filler', subfolder), tablet)
        subfolder_tiff_path = os.path.join(general_tiff_path.replace('filler', subfolder), tablet)
        
        # Ensure TIFF output directory exists
        os.makedirs(subfolder_tiff_path, exist_ok=True)
        
        # Get list of all .mhd files in the subfolder
        for file_name in os.listdir(subfolder_mhd_path):
            if file_name.endswith('.mhd'):
                print(f"Found .mhd file: {file_name} in {subfolder}")
                mhd_file = os.path.join(subfolder_mhd_path, file_name)
                
                try:
                    # Load and convert the .mhd file
                    image = sitk.ReadImage(mhd_file)
                    print(f"Loaded image: {mhd_file}, size: {image.GetSize()}")
                    image_array = sitk.GetArrayFromImage(image)  # Convert to numpy array
                    print(f"Image array shape: {image_array.shape}")
                    print(f"Min: {np.min(image_array)}, Max: {np.max(image_array)}")
                    
                    # Normalize for segmented images (Microsphere: [0:2], Tablet: [0:4], Spray Dried Dispersion: [0:4])
                    if subfolder == "segmented":
                        if np.max(image_array) > 0:  # Avoid division by zero
                            image_array = (image_array / np.max(image_array) * 255).astype(np.uint8) # Converts the segmentation labels into visualizable grayscale colors
                        else:
                            print(f"Warning: {mhd_file} is completely black.")
                    
                    # Save each slice as a TIFF file
                    base_name = os.path.splitext(file_name)[0]
                    for i, slice_array in enumerate(image_array):
                        slice_image = Image.fromarray(slice_array)
                        tiff_file = os.path.join(subfolder_tiff_path, f"{base_name}_slice_{i:04d}.tiff")
                        slice_image.save(tiff_file)
                        print(f"Saved TIFF: {tiff_file}")
                except Exception as e:
                    print(f"Error processing {mhd_file}: {e}")

print("All conversions complete!")

Found .mhd file: D1539_stitch-A_Stitch-Crop_ml_seg.mhd in segmented
Loaded image: D:\Darren\Files\database\tablet_dataset\segmented\mhd\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg.mhd, size: (1541, 1541, 731)
Image array shape: (731, 1541, 1541)
Min: 0, Max: 2
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg_slice_0000.tiff
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg_slice_0001.tiff
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg_slice_0002.tiff
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg_slice_0003.tiff
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitch-Crop_ml_seg_slice_0004.tiff
Saved TIFF: D:\Darren\Files\database\tablet_dataset\segmented\tiff\6_30pWTMCC\D1539_stitch-A_Stitc